In [26]:
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from cleaning_function import *
from sklearn.model_selection import train_test_split
data = pd.read_csv('no_race_dataset.csv')

In [35]:
data

,Unnamed: 0,statecounty_x,"AirPollutant: 1,3-butadiene_ug/m^3_EPA2018",AirPollutant: Acetaldehyde_ug/m^3_EPA2017,AirPollutant: Acetaldehyde_ug/m^3_EPA2018,AirPollutant: Benzene_ug/m^3_EPA2017,AirPollutant: Benzene_ug/m^3_EPA2018,AirPollutant: Carbon tetrachloride_ug/m^3_EPA2017,AirPollutant: Carbon tetrachloride_ug/m^3_EPA2018,AirPollutant: Formaldehyde_ug/m^3_EPA2017,...,"Expected Population 2020, total, ages 55-59","Expected Population 2020, total, ages 60-64","Expected Population 2020, total, ages 65-69","Expected Population 2020, total, ages 70-74","Expected Population 2020, total, ages 75-79",Code,State,County [2],County Seat(s) [3],Population (2010)
0,0,01autauga,0.01,1.50,1.41,0.19,0.18,0.51,0.37,1.84,...,4085.46,3483.00,2628.37,2183.59,1620.94,01autauga,AL,Autauga,Prattville,54571.0
1,1,01baldwin,0.01,1.15,1.10,0.17,0.16,0.51,0.37,1.47,...,12612.46,12158.07,10783.76,9674.46,7689.06,01baldwin,AL,Baldwin,Bay Minette,182265.0
2,2,01barbour,0.02,1.51,1.39,0.18,0.17,0.50,0.37,1.83,...,2003.35,1719.75,1536.10,1445.48,943.48,01barbour,AL,Barbour,Clayton,27457.0
3,3,01bibb,0.01,1.61,1.54,0.18,0.18,0.51,0.37,1.90,...,1675.91,1412.47,1136.45,1023.28,643.26,01bibb,AL,Bibb,Centreville,22915.0
4,4,01blount,0.01,1.24,1.19,0.20,0.19,0.50,0.37,1.65,...,3884.37,3472.96,3117.66,2815.41,2099.01,01blount,AL,Blount,Oneonta,57322.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3138,3138,56sweetwater,0.00,0.32,0.29,0.10,0.10,0.42,0.31,0.51,...,2870.24,3202.31,2723.26,1797.71,997.16,56sweetwater,WY,Sweetwater,Green River,43806.0
3139,3139,56teton,0.00,0.37,0.30,0.07,0.07,0.41,0.30,0.45,...,1490.07,1535.49,1449.62,1148.10,760.06,56teton,WY,Teton,Jackson,21294.0
3140,3140,56uinta,0.00,0.30,0.27,0.08,0.08,0.41,0.30,0.50,...,1367.07,1615.53,1353.29,854.44,533.69,56uinta,WY,Uinta,Evanston,21118.0
3141,3141,56washakie,0.00,0.41,0.34,0.09,0.10,0.45,0.33,0.54,...,563.92,600.18,568.73,461.20,380.78,56washakie,WY,Washakie,Worland,8533.0


In [27]:
data.shape

(3143, 504)

In [28]:
include_columns(data, ['outcome'])
y = data['HealthOutcome_percent_pop_with_obesity_Places_BRFSS_2021']
x = exclude_columns(data, ['HealthOutcome_percent_pop_with_obesity_Places_BRFSS_2021'])
x = x.select_dtypes(include = ['number'])


In [29]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_standardized = scaler.fit_transform(x)



In [36]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error

# Define model configurations with more options
configs = [
    {"hidden_units": [128, 64], "batch_size": 32, "epochs": 50, "dropout_rate": 0.2},
    {"hidden_units": [256, 128], "batch_size": 64, "epochs": 50, "dropout_rate": 0.3},
    {"hidden_units": [64, 32], "batch_size": 32, "epochs": 100, "dropout_rate": 0.1},
    {"hidden_units": [512, 256], "batch_size": 128, "epochs": 50, "dropout_rate": 0.4},
    {"hidden_units": [64, 128], "batch_size": 16, "epochs": 75, "dropout_rate": 0.25},
    {"hidden_units": [128, 128], "batch_size": 64, "epochs": 200, "dropout_rate": 0.5},
    {"hidden_units": [256, 64, 32], "batch_size": 32, "epochs": 150, "dropout_rate": 0.3},
    {"hidden_units": [128, 64, 32], "batch_size": 16, "epochs": 100, "dropout_rate": 0.4},
    {"hidden_units": [512, 256], "batch_size": 32, "epochs": 50, "dropout_rate": 0.35},
    {"hidden_units": [256, 128, 64], "batch_size": 64, "epochs": 200, "dropout_rate": 0.2},
    {"hidden_units": [128, 64], "batch_size": 128, "epochs": 25, "dropout_rate": 0.15},
    {"hidden_units": [64, 32], "batch_size": 32, "epochs": 300, "dropout_rate": 0.2},
]

# Split the data into training, validation, and test sets
X_train, X_test, y_train, y_test = train_test_split(X_standardized, y, test_size=0.2, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

best_model = None
best_test_loss = float('inf')
best_test_mae = float('inf')
best_config = None  # To store the best configuration

# Loop through different model configurations
for config in configs:
    # Step 2: Build the neural network model with the current configuration
    model = Sequential()
    model.add(Dense(config["hidden_units"][0], input_dim=X_train.shape[1], activation='relu'))
    model.add(Dropout(config["dropout_rate"]))  # Dynamic dropout rate
    model.add(Dense(config["hidden_units"][1], activation='relu'))
    
    # Add additional hidden layers if there are more than 2 hidden layers in the configuration
    if len(config["hidden_units"]) > 2:
        for units in config["hidden_units"][2:]:
            model.add(Dense(units, activation='relu'))
            model.add(Dropout(config["dropout_rate"]))  # Dynamic dropout rate
    
    # Output layer
    model.add(Dense(1, activation='linear'))  # Output layer
    
    # Compile the model using Mean Squared Error as the loss function and Mean Absolute Error as a metric
    model.compile(loss='mse', optimizer='adam', metrics=['mae'])
    
    # Train the model using the training data and validate using the validation data
    model.fit(X_train, y_train, epochs=config["epochs"], batch_size=config["batch_size"], validation_data=(X_val, y_val), verbose=0)  # Suppress epoch output
    
    # Step 4: Evaluate the model using the test data
    test_loss, test_mae = model.evaluate(X_test, y_test)

    # Print the final test loss and MAE for this configuration
    print(f"Config: {config} - Test Loss: {test_loss}, Test MAE: {test_mae}")
    
    # Update the best model if the current model is better
    if test_loss < best_test_loss:
        best_test_loss = test_loss
        best_test_mae = test_mae
        best_model = model
        best_config = config  # Store the configuration of the best model

# Print the results for the best model
print(f"\nBest Test Loss: {best_test_loss}, Best Test MAE: {best_test_mae}")
print(f"Best Configuration: {best_config}")

# Optionally, you can now use `best_model` to make predictions on new data or further analysis


/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 9.5147 - mae: 2.2615  
Config: {'hidden_units': [128, 64], 'batch_size': 32, 'epochs': 50, 'dropout_rate': 0.2} - Test Loss: 8.648736000061035, Test MAE: 2.254458427429199


/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 9.6840 - mae: 2.3704  
Config: {'hidden_units': [256, 128], 'batch_size': 64, 'epochs': 50, 'dropout_rate': 0.3} - Test Loss: 9.215559005737305, Test MAE: 2.3572428226470947


/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


...

20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 8.6331 - mae: 2.1742
Config: {'hidden_units': [256, 128, 64], 'batch_size': 64, 'epochs': 200, 'dropout_rate': 0.2} - Test Loss: 8.73620319366455, Test MAE: 2.2098865509033203


/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 11.6403 - mae: 2.6633 
Config: {'hidden_units': [128, 64], 'batch_size': 128, 'epochs': 25, 'dropout_rate': 0.15} - Test Loss: 11.774846076965332, Test MAE: 2.6346306800842285


/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 11.7244 - mae: 2.7966 
Config: {'hidden_units': [64, 32], 'batch_size': 32, 'epochs': 300, 'dropout_rate': 0.2} - Test Loss: 11.358819961547852, Test MAE: 2.7668797969818115

Best Test Loss: 7.045512676239014, Best Test MAE: 2.047248601913452
Best Configuration: {'hidden_units': [256, 64, 32], 'batch_size': 32, 'epochs': 150, 'dropout_rate': 0.3}


Shape of y_test: (629,)
Shape of predictions: (3143, 1)
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
Mean Squared Error: 11.204610311553543
Mean Absolute Error: 2.5065892061557986
